## Instalações e imports:

In [33]:
!pip install selenium
!pip install unidecode

In [34]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
from google.colab import files
import pandas as pd
import csv
import time

## Web Scraper Gupy:

In [45]:
# Palavras-chave para a pesquisa
pesquisa = "dados"

# Caso driver pare antes - aumentar o tempo de pausa (tempo de execução é impactado mas garante resultado uniforme)
SCROLL_PAUSE_TIME = 2

In [47]:
# Configurações do webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)
url = f'https://portal.gupy.io/job-search/term={pesquisa}'

driver.get(url)

# Driver realiza leitura da página - Vagas aparecem de acordo com o scroll do usuário
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    # se parar de scrollar para baixo, sair do loop
    if new_height == last_height:
        break
    last_height = new_height

html_content = driver.page_source

# Beautifulsoup para separar os valores a obter somente o que é relevante da página
soup = BeautifulSoup(html_content, 'html.parser')
blocks = soup.find_all('div', class_='sc-a3bd7ea-0 HCzvP')
# UTC+0
current_time = time.strftime("%d-%m-%Y")
csv_filename = f'vagas_gupy_{pesquisa}_{current_time}.csv'

with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Empresa', 'Cargo', 'Local', 'Data publicacao', 'Tipo de cargo', 'Remoto/Presencial'])

    for block in blocks:
        # Valores padrão encontrados nos blocos de vaga - source code da página
        company_name_elem = block.select_one('p.sc-efBctP.dpAAMR.sc-a3bd7ea-6.cQyvth')
        job_title_elem = block.select_one('h2')
        location_elem = block.find('span', class_='sc-23336bc7-1 cezNaf')
        published_date_elem = block.find('div', class_="sc-1db88588-1 joOyhp")
        span_elems = block.find_all('span', class_='sc-23336bc7-1 cezNaf')

        # Verifica se há texto para todos os elementos, se não tiver, escreve N/A
        company_name = unidecode(company_name_elem.text.strip()) if company_name_elem else "N/A"
        job_title = unidecode(job_title_elem.text.strip()) if job_title_elem else "N/A"
        location = unidecode(location_elem.text.strip()) if location_elem else "N/A"
        published_date_text = published_date_elem.find('p').text.strip() if published_date_elem else "N/A"
        published_date = published_date_text.replace("Publicada em: ", "")
        span_values = [unidecode(span.text.strip()) for span in span_elems]

        # Escreve linhas no CSV
        row_data = [company_name, job_title, location, published_date, span_values[2], span_values[1]]
        csv_writer.writerow(row_data)

driver.quit()

files.download(csv_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [52]:
#Visualiza os 5 primeiros valores do CSV
df = pd.read_csv(csv_filename)
df.head()

,Empresa,Cargo,Local,Data publicacao,Tipo de cargo,Remoto/Presencial
0,LEAN SALES,Assistente De Dados,Itajai - SC,16/08/2023,Efetivo,Presencial
1,NAVA | Technology for Business,PESSOA ESPECIALISTA DE GOVERNANCA DE DADOS,Sao Paulo - SP,16/08/2023,Efetivo,Presencial
2,Daniel Advogados,Cientista de Dados,Rio de Jan... - RJ,16/08/2023,Efetivo,Presencial
3,NAVA | Technology for Business,ANALISTA DLP e PROTECAO DE DADOS SR,Osasco - SP,16/08/2023,Efetivo,Presencial
4,NAVA | Technology for Business,ANALISTA DLP e PROTECAO DE DADOS SR,Osasco - SP,16/08/2023,Efetivo,Presencial


## Benchmark de performance sem Selenium:

In [51]:
# Webscraping sem Selenium (acessa somente as vagas carregadas com a página) - teste de performance

pesquisa = "dados"


url = f'https://portal.gupy.io/job-search/term={pesquisa}'
response = requests.get(url)
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')

blocks = soup.find_all('div', class_='sc-a3bd7ea-0 HCzvP')

for block in blocks:
  company_name_elem = block.select_one('p.sc-efBctP.dpAAMR.sc-a3bd7ea-6.cQyvth')
  job_title_elem = block.select_one('h2')
  location_elem = block.find('span', class_='sc-23336bc7-1 cezNaf')
  published_date_elem = block.find('div', class_="sc-1db88588-1 joOyhp")
  span_elems = block.find_all('span', class_='sc-23336bc7-1 cezNaf')

  company_name = unidecode(company_name_elem.text.strip()) if company_name_elem else "N/A"
  job_title = unidecode(job_title_elem.text.strip()) if job_title_elem else "N/A"
  location = unidecode(location_elem.text.strip()) if location_elem else "N/A"
  published_date_text = published_date_elem.find('p').text.strip() if published_date_elem else "N/A"
  published_date = published_date_text.replace("Publicada em: ", "")
  span_values = [unidecode(span.text.strip()) for span in span_elems]

  print("Empresa:", company_name)
  print("Cargo:", job_title)
  print("Localização:", location)
  print(published_date)
  print("Tipo de cargo:", span_values[2])
  print("Remoto ou presencial:", span_values[1])
  print()



Empresa: LEAN SALES
Cargo: Assistente De Dados
Localização: Itajai - SC
16/08/2023
Tipo de cargo: Efetivo
Remoto ou presencial: Presencial

Empresa: NAVA | Technology for Business
Cargo: PESSOA ESPECIALISTA DE GOVERNANCA DE DADOS
Localização: Sao Paulo - SP
16/08/2023
Tipo de cargo: Efetivo
Remoto ou presencial: Presencial

Empresa: Daniel Advogados
Cargo: Cientista de Dados
Localização: Rio de Jan... - RJ
16/08/2023
Tipo de cargo: Efetivo
Remoto ou presencial: Presencial

Empresa: NAVA | Technology for Business
Cargo: ANALISTA  DLP e PROTECAO DE DADOS SR
Localização: Osasco - SP
16/08/2023
Tipo de cargo: Efetivo
Remoto ou presencial: Presencial

Empresa: NAVA | Technology for Business
Cargo: ANALISTA  DLP e PROTECAO DE DADOS SR
Localização: Osasco - SP
16/08/2023
Tipo de cargo: Efetivo
Remoto ou presencial: Presencial

Empresa: NAVA | Technology for Business
Cargo: ANALISTA DE PROTECAO DE DADOS SR - DISCOVERY, PROTECAO E CRIPTOGRAFIA
Localização: Osasco - SP
16/08/2023
Tipo de cargo: 